In [ ]:
# imports
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

plt.rcParams['figure.figsize'] = [20, 10]

In [ ]:
def reject_outliers(data: list, m = 5.):
    data = np.asarray(data)
    d = np.abs(data - np.median(data))
    mdev = np.median(d)
    s = d/mdev if mdev else np.zeros(len(d))
    return data[s<m]

In [ ]:
def _calculate_user_timestamp_diffs(original_set: pd.DataFrame, simulated_set: pd.DataFrame) -> list[list]:

    out = []

    # create differences for each user
    for user in range(1, 32):
        original_user_data = original_set[original_set['user'] == user]
        simulated_user_data = simulated_set[simulated_set['user'] == user]

        assert len(original_user_data) == len(simulated_user_data)

        # assert 15 sets
        assert len(original_user_data['set'].unique()) == len(simulated_user_data['set'].unique()) == 15

        diffs = [None] * len(original_user_data)
        assert len(diffs) == len(original_user_data) == len(simulated_user_data)


        for (i , (original, simulated)) in enumerate(zip(original_user_data['timestamp'], simulated_user_data['timestamp'])):
            diff = abs(original - simulated)
            diffs[i] = diff

        assert len(diffs) == len(original_user_data) == len(simulated_user_data)
        assert not any(elem is None for elem in diffs)

        out.append(diffs)

    assert len(out) == 31


    return out


In [ ]:
def calculate_user_distances_diffs(original_set: pd.DataFrame, simulated_set: pd.DataFrame) -> list[list]:

    out = []

    # create differences for each user
    for user in range(1, 32):
        original_user_data = original_set[original_set['user'] == user]
        simulated_user_data = simulated_set[simulated_set['user'] == user]

        assert len(original_user_data) == len(simulated_user_data)

        # assert 15 sets
        assert len(original_user_data['set'].unique()) == len(simulated_user_data['set'].unique()) == 15

        diffs = [None] * len(original_user_data)
        assert len(diffs) == len(original_user_data) == len(simulated_user_data)


        for (i , (original, simulated)) in enumerate(zip(original_user_data['distance'], simulated_user_data['distance'])):
            diff = abs(original - simulated)
            diffs[i] = diff

        assert len(diffs) == len(original_user_data) == len(simulated_user_data)
        assert not any(elem is None for elem in diffs)

        out.append(diffs)

    assert len(out) == 31


    return out

In [ ]:
# visualize
def visualize_hist(data: list[list], browser:str, out_file: str):
    data = [[x / 1000 for x in set] for set in data]
    for (i, d) in enumerate(data):
        bef = len(d)
        d = reject_outliers(d)
        plt.hist(d, label= f'User{i + 1}', alpha=1)
        
    plt.title(f'Differences in distances of input Event between original dataset and simulated dataset ({browser})', fontsize=18)
    plt.xlabel('Difference to last Event (sec)', fontsize=18)
    plt.ylabel('Occurences per User', fontsize=18)
    plt.tight_layout()
    plt.legend()
    plt.savefig(out_file, bbox_inches="tight", pad_inches=0.3)
    plt.show()

def visualize_plot(data: list[list], browser:str, out_file: str):
    data = [[x/ 1000 for x in set] for set in data]
    for (i, d) in enumerate(data):
        d = reject_outliers(d)
        plt.plot(d, label= f'User{i + 1}', alpha=1)
        
    plt.title(f'Differences in distances of input Event between original dataset and simulated dataset ({browser})', fontsize=18)
    plt.xlabel('Input Event (all sets)', fontsize=18)
    plt.ylabel('Difference to last Event (sec)', fontsize=18)
    plt.tight_layout()
    plt.legend()
    plt.savefig(out_file, bbox_inches="tight", pad_inches=0.3)
    plt.show()

def visualize_boxplot(data: list[list], browser:str, out_file: str):
    data = [[x / 1000 for x in set] for set in data]
    plt.boxplot(data, positions = range(31), showfliers=False, labels = [f'User{i}' for i in range(1,32)])

    plt.title(f'Differences in distances of input Event between original dataset and simulated dataset ({browser})', fontsize=18)
    plt.xlabel('Nutzer', fontsize=18)
    plt.ylabel('Difference to last Event (sec)', fontsize=18)
    plt.tight_layout()
    plt.savefig(out_file, bbox_inches="tight", pad_inches=0.3)
    plt.show()

In [ ]:
# read in original
original = pd.read_csv('./FreeText-Dataset-31-USERS.csv')

labels = [f'User{i}' for i in range(1,32)]
positions = range(31)

In [ ]:
# chrome isolated
chrome_isolated = pd.read_csv('./chrome/isolated/complete.csv')
chrome_isolated_user_diffs = calculate_user_distances_diffs(original, chrome_isolated)

#visualize_hist(chrome_isolated_user_diffs, "Chrome Isolated", "./chrome/isolated/hist.jpeg")
#visualize_plot(chrome_isolated_user_diffs, "Chrome Isolated", "./chrome/isolated/plot.jpeg")
visualize_boxplot(chrome_isolated_user_diffs, "Chrome Isolated", "./chrome/isolated/box.jpeg")

In [ ]:
# chrome unisolated
chrome_unisolated = pd.read_csv('./chrome/unisolated/complete.csv')
chrome_unisolated_user_diffs = calculate_user_distances_diffs(original, chrome_unisolated)

#visualize_hist(chrome_unisolated_user_diffs, "Chrome Unisolated", "./chrome/unisolated/hist.jpeg")
#visualize_plot(chrome_unisolated_user_diffs, "Chrome Unisolated", "./chrome/unisolated/hist.jpeg")
visualize_boxplot(chrome_unisolated_user_diffs, "Chrome Unisolated", "./chrome/unisolated/box.jpeg")

In [ ]:
# chromium isolated
chromium_isolated = pd.read_csv('./chromium/isolated/complete.csv')
chromium_isolated_user_diffs = calculate_user_distances_diffs(original, chromium_isolated)

#visualize_hist(chromium_isolated_user_diffs, "Chromium Isolated", "./chromium/isolated/hist.jpeg")
#visualize_plot(chromium_isolated_user_diffs, "Chromium Isolated", "./chromium/isolated/hist.jpeg")
visualize_boxplot(chromium_isolated_user_diffs, "Chromium Isolated", "./chromium/isolated/box.jpeg")

In [ ]:
# chromium unisolated
chromium_unisolated = pd.read_csv('./chromium/unisolated/complete.csv')
chromium_unisolated_user_diffs = calculate_user_distances_diffs(original, chromium_unisolated)

#visualize_hist(chromium_unisolated_user_diffs, "Chromium Unisolated", "./chromium/unisolated/hist.jpeg")
#visualize_plot(chromium_unisolated_user_diffs, "Chromium Unisolated", "./chromium/unisolated/hist.jpeg")
visualize_boxplot(chromium_unisolated_user_diffs, "Chromium Unisolated", "./chromium/unisolated/box.jpeg")

In [ ]:
# edge isolated
edge_isolated = pd.read_csv('./edge/isolated/complete.csv')
edge_isolated_user_diffs = calculate_user_distances_diffs(original, edge_isolated)

#visualize_hist(edge_isolated_user_diffs, "Edge Isolated", "./edge/isolated/hist.jpeg")
#visualize_plot(edge_isolated_user_diffs, "Edge Isolated", "./edge/isolated/hist.jpeg")
visualize_boxplot(edge_isolated_user_diffs, "Edge Isolated", "./edge/isolated/box.jpeg")

In [ ]:
# edge unisolated
edge_unisolated = pd.read_csv('./edge/unisolated/complete.csv')
edge_unisolated_user_diffs = calculate_user_distances_diffs(original, edge_unisolated)

visualize_hist(edge_unisolated_user_diffs, "Edge Unisolated", "./edge/unisolated/hist.jpeg")
visualize_plot(edge_unisolated_user_diffs, "Edge Unisolated", "./edge/unisolated/hist.jpeg")
visualize_boxplot(edge_unisolated_user_diffs, "Edge Unisolated", "./edge/unisolated/box.jpeg")

In [ ]:
# tor unisolated
tor_unisolated = pd.read_csv('./tor/unisolated/complete.csv')
tor_unisolated_user_diffs = calculate_user_distances_diffs(original, tor_unisolated)

#visualize_hist(tor_unisolated_user_diffs, "Tor Unisolated", "./tor/unisolated/hist.jpeg")
#visualize_plot(tor_unisolated_user_diffs, "Tor Unisolated", "./tor/unisolated/hist.jpeg")
visualize_boxplot(tor_unisolated_user_diffs, "Tor Unisolated", "./tor/unisolated/box.jpeg")

In [ ]:
# firefox isolated
firefox_isolated = pd.read_csv('./firefox/isolated/complete.csv')
firefox_isolated_user_diffs = calculate_user_distances_diffs(original, firefox_isolated)

visualize_hist(firefox_isolated_user_diffs, "Firefox Isolated", "./firefox/isolated/hist.jpeg")
visualize_plot(firefox_isolated_user_diffs, "Firefox Isolated", "./firefox/isolated/hist.jpeg")
visualize_boxplot(firefox_isolated_user_diffs, "Firefox Isolated", "./firefox/isolated/box.jpeg")

In [ ]:
# firefox unisolated
firefox_unisolated = pd.read_csv('./firefox/unisolated/complete.csv')
firefox_unisolated_user_diffs = calculate_user_distances_diffs(original, firefox_unisolated)

#visualize_hist(firefox_unisolated_user_diffs, "Firefox Unisolated", "./firefox/unisolated/hist.jpeg")
#visualize_plot(firefox_unisolated_user_diffs, "Firefox Unisolated", "./firefox/unisolated/hist.jpeg")
visualize_boxplot(firefox_unisolated_user_diffs, "Firefox Unisolated", "./firefox/unisolated/box.jpeg")

In [ ]:
# firefox isolated resistFP
firefox_resistFP_isolated = pd.read_csv('./firefox_rf/isolated/complete.csv')
firefox_resistFP_isolated_user_diffs = calculate_user_distances_diffs(original, firefox_resistFP_isolated)

#visualize_hist(firefox_resistFP_isolated_user_diffs, "Firefox RFP Isolated", "./firefox_rf/isolated/hist.jpeg")
#visualize_plot(firefox_resistFP_isolated_user_diffs, "Firefox RFP Isolated", "./firefox_rf/isolated/hist.jpeg")
visualize_boxplot(firefox_resistFP_isolated_user_diffs, "Firefox RFP Isolated", "./firefox_rf/isolated/box.jpeg")

In [ ]:
# firefox unisolated resistFP
firefox_resistFP_unisolated = pd.read_csv('./firefox_rf/unisolated/complete.csv')
firefox_resistFP_unisolated_user_diffs = calculate_user_distances_diffs(original, firefox_resistFP_unisolated)

#visualize_hist(firefox_resistFP_unisolated_user_diffs, "Firefox RFP Unisolated", "./firefox_rf/unisolated/hist.jpeg")
#visualize_plot(firefox_resistFP_unisolated_user_diffs, "Firefox RFP Unisolated", "./firefox_rf/unisolated/hist.jpeg")
visualize_boxplot(firefox_resistFP_unisolated_user_diffs, "Firefox RFP Unisolated", "./firefox_rf/unisolated/box.jpeg")